<a href="https://colab.research.google.com/github/MarkLo127/stockinfo/blob/main/datatest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 系統相關與時間操作
from posixpath import expanduser
from datetime import datetime, timedelta
import time

# 資料分析
import pandas as pd  # 資料處理

# 資料擷取與網路相關
import yfinance as yf  # 股票數據
import requests as res  # HTTP 請求
from bs4 import BeautifulSoup  # 網頁解析
import re  # 正規表示式處理

# 地理與地圖相關
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderInsufficientPrivileges
import folium

# 畫圖相關
import plotly
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio
import plotly.express as px

# 財務技術分析
from ta.trend import MACD  # MACD 技術指標
from ta.momentum import StochasticOscillator, RSIIndicator  # 隨機震盪與 RSI 技術指標

1.大盤指數

In [ ]:
def plot_financial_data(period, time, plot_type='index'):
    # Fetch historical data for the indexes
    symbols = {
        'index': ['^IXIC', '^VIX', '^GSPC', '^DJI', '^SOX', '^RUT'],
        'foreign': ['^GSPC', '^IXIC', '^HSI', '399001.SZ', '^TWII', '^N225']
    }

    data = {}

    for symbol in symbols[plot_type]:
        data[symbol] = yf.download(symbol, period=period)['Close']

    if plot_type == 'index':
        print(f'美股大盤＆中小企業{time}走勢')

        # Create Plotly subplot figure for indexes
        fig = make_subplots(rows=3, cols=2, subplot_titles=["NASDAQ", "VIX", "S&P 500", "DJIA", "PHLX Semiconductor Sector", "Russell-2000"])

        for i, symbol in enumerate(symbols['index']):
            fig.add_trace(go.Scatter(x=data[symbol].index, y=data[symbol].values, mode='lines', name=symbol), row=(i // 2) + 1, col=(i % 2) + 1)

        fig.update_layout(height=800, width=1000, showlegend=False)
        fig.show()

    elif plot_type == 'foreign':
        print(f'美股大盤＆海外大盤{time}走勢')

        # Create Plotly subplot figure for foreign indexes
        fig = make_subplots(rows=3, cols=2, subplot_titles=["S&P 500", "NASDAQ", "恆生指數", "深證成份指數", "台灣加權指數", "日經225指數"])

        for i, symbol in enumerate(symbols['foreign']):
            row = (i // 2) + 1
            col = (i % 2) + 1

            # Apply conversion for foreign indexes
            if symbol in ['^HSI', '399001.SZ', '^TWII', '^N225']:
                conversion_factor = {'^HSI': 0.1382, '399001.SZ': 0.1382, '^TWII': 0.0308, '^N225': 0.0064}
                fig.add_trace(go.Scatter(x=data[symbol].index, y=(data[symbol] * conversion_factor[symbol]).values, mode='lines', name=symbol), row=row, col=col)
            else:
                fig.add_trace(go.Scatter(x=data[symbol].index, y=data[symbol].values, mode='lines', name=symbol), row=row, col=col)

        fig.update_layout(height=800, width=1000, showlegend=False)
        fig.show()

# Example usage
period = 'ytd'
time = '年初至今'
plot_financial_data(period, time, plot_type='index')  # For US indexes
plot_financial_data(period, time, plot_type='foreign')  # For foreign indexes

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


美股大盤＆中小企業年初至今走勢


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

美股大盤＆海外大盤年初至今走勢


2.公司基本資訊

In [ ]:
def get_company_details_and_display(symbol):
    try:
        # 獲取公司基本資訊
        stock_info = yf.Ticker(symbol)
        com_info = stock_info.info

        if com_info:
            selected_indicators = [
                'longName', 'country', 'city', 'marketCap', 'totalRevenue',
                'grossMargins', 'operatingMargins', 'profitMargins', 'trailingEps',
                'pegRatio', 'dividendRate', 'payoutRatio', 'bookValue',
                'operatingCashflow', 'freeCashflow', 'returnOnEquity'
            ]

            selected_info = {indicator: com_info.get(indicator, '') for indicator in selected_indicators}

            # 建立字典翻譯
            translation = {
                'longName': '公司名稱', 'country': '國家', 'city': '城市',
                'marketCap': '市值', 'totalRevenue': '總收入', 'grossMargins': '毛利率',
                'operatingMargins': '營業利潤率', 'profitMargins': '净利率',
                'trailingEps': '每股收益', 'pegRatio': 'PEG 比率',
                'dividendRate': '股息率', 'payoutRatio': '股息支付比例',
                'bookValue': '每股淨資產', 'operatingCashflow': '營運現金流',
                'freeCashflow': '自由現金流', 'returnOnEquity': '股東權益報酬率'
            }

            # Pandas DataFrame
            company_info = pd.DataFrame.from_dict(selected_info, orient='index', columns=['Value'])
            company_info.rename(index=translation, inplace=True)

            # 轉換成百分比
            percent_columns = ['毛利率', '營業利潤率', '净利率', '股息率', '股息支付比例', '股東權益報酬率']
            for col in percent_columns:
                if col in company_info.index:
                    company_info.at[col, 'Value'] = pd.to_numeric(company_info.at[col, 'Value'], errors='coerce')  # 将非数字转换为 NaN
                    company_info.at[col, 'Value'] = f"{company_info.at[col, 'Value']:.2%}" if pd.notna(company_info.at[col, 'Value']) else None

            # 千分位表示
            company_info['Value'] = company_info['Value'].apply(lambda x: "{:,.0f}".format(x) if isinstance(x, (int, float)) and x >= 1000 else x)
            print("公司基本資訊:")
            print(company_info)

            print("公司位置資訊:")
            display_location(com_info)

        else:
            print(f"{symbol} 公司的基本資訊無法獲取。")

    except Exception as e:
        print(f"無法獲取公司基本資訊：{str(e)}")

def display_location(com_info):
    if 'city' in com_info and 'country' in com_info:
        city = com_info['city']
        country = com_info['country']

        try:
            # 使用 Nominatim 服務進行地理編碼
            geolocator = Nominatim(user_agent="myGeocoder")
            location = geolocator.geocode(f"{city}, {country}")

            if location:
                # 使用 Folium 創建地圖，並將其定位到公司位置
                map = folium.Map(location=[location.latitude, location.longitude], zoom_start=10)
                # 添加標記
                folium.Marker([location.latitude, location.longitude], popup=f"{city}, {country}").add_to(map)
                return map
            else:
                print("無法找到公司位置")

        except GeocoderInsufficientPrivileges as e:
            print(f"地理編碼服務無法訪問: {str(e)}")
    else:
        print("缺少城市或國家")

# 使用 NVDA 呼叫該函數
symbol = 'NVDA'
get_company_details_and_display(symbol)

公司基本資訊:
                      Value
公司名稱     NVIDIA Corporation
國家            United States
城市              Santa Clara
市值        3,102,841,241,600
總收入          96,307,003,392
毛利率                  75.98%
營業利潤率                62.06%
净利率                  55.04%
每股收益                   2.14
PEG 比率                 0.83
股息率                   4.00%
股息支付比例                1.03%
每股淨資產                 2.368
營運現金流        48,663,998,464
自由現金流        33,725,874,176
股東權益報酬率             123.77%
公司位置資訊:
地理編碼服務無法訪問: Non-successful status code 403


3.公司經營狀況



In [ ]:
import requests as res
import pandas as pd
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def get_stock_details_and_plot(symbol):
    # Step 1: Get stock statistics
    url = f"https://finviz.com/quote.ashx?t={symbol}&p=d#statements"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}

    try:
        response = res.get(url, headers=headers)
        response.raise_for_status()
    except res.exceptions.RequestException as e:
        print(f"獲取 {symbol} 數據時出錯: {e}")
        return None

    # Step 2: Parse the HTML to get the data
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', class_='snapshot-table2')
    if not table:
        print("頁面上未找到表格")
        return None

    # Step 3: Extract data into a dictionary
    rows = table.find_all('tr')
    data = {}
    for row in rows:
        cells = row.find_all('td')
        for i in range(0, len(cells), 2):
            key = cells[i].get_text(strip=True)
            value = cells[i + 1].get_text(strip=True)
            data[key] = value

    # Step 4: Process values for categorization and plotting
    def process_value(value):
        if isinstance(value, str):
            value = value.replace(',', '')  # Remove commas for thousands
            if value.endswith('%'):
                return float(value[:-1])  # Convert percentage to float
            elif value.endswith('B'):
                return float(value[:-1]) * 1e9  # Convert billions to float
            elif value.endswith('M'):
                return float(value[:-1]) * 1e6  # Convert millions to float
            elif value.endswith('K'):
                return float(value[:-1]) * 1e3  # Convert thousands to float
            elif value.replace('.', '', 1).isdigit():  # Check if it's a numeric string
                return float(value)  # Convert numeric string to float
        return value  # Return the original value if no conversion is needed

    # Create a DataFrame for categorization
    df = pd.DataFrame(list(data.items()), columns=['Metric', 'Value'])

    # Step 5: Categorize and plot data
    categories = {
        '估值指標': ['P/E', 'Forward P/E', 'PEG', 'P/S', 'P/B', 'P/C', 'P/FCF'],
        '盈利能力': ['Gross Margin', 'Oper. Margin', 'Profit Margin', 'ROA', 'ROE', 'ROI'],
        '表現指標': ['Perf Week', 'Perf Month', 'Perf Quarter', 'Perf Half Y', 'Perf Year', 'Perf YTD'],
        '流動性': ['Quick Ratio', 'Current Ratio'],
        '所有權': ['Insider Own', 'Inst Own', 'Shs Outstanding'],
        '銷售與收入': ['Sales', 'Income'],
        '簡單移動平均': ['SMA20', 'SMA50', 'SMA200'],
        '其他': ['EPS (ttm)', 'EPS next Y', 'EPS next Q', 'Book/sh', 'Cash/sh', 'Dividend', 'Dividend %', 'Beta']
    }

    specs = [
        [{'type': 'xy'}, {'type': 'xy'}],
        [{'type': 'xy'}, {'type': 'xy'}],
        [{'type': 'domain'}, {'type': 'domain'}],
        [{'type': 'xy'}, {'type': 'xy'}]
    ]

    fig = make_subplots(rows=4, cols=2, subplot_titles=list(categories.keys()), specs=specs)
    plot_idx = 0

    for category, metrics in categories.items():
        plot_idx += 1
        row = (plot_idx - 1) // 2 + 1
        col = (plot_idx - 1) % 2 + 1
        cat_data = df[df['Metric'].isin(metrics)].copy()
        cat_data['Value'] = cat_data['Value'].apply(process_value)
        cat_data['Value'] = pd.to_numeric(cat_data['Value'], errors='coerce')  # Convert non-numeric values to NaN
        cat_data = cat_data.dropna(subset=['Value'])  # Drop rows with NaN values in 'Value'
        cat_data = cat_data.sort_values(by='Value', ascending=False)

        if category in ['所有權', '銷售與收入']:
            chart = go.Pie(labels=cat_data['Metric'], values=cat_data['Value'], name=category, sort=False)
        else:
            chart = go.Bar(x=cat_data['Metric'], y=cat_data['Value'], name=category, marker=dict(color=cat_data['Value'], colorscale='Viridis'))

        fig.add_trace(chart, row=row, col=col)

    fig.update_layout(height=1200, showlegend=True)
    print(f'{symbol}-基本資訊')
    fig.show()  # Use fig.show() to display the plot

# Example usage
symbol = 'NVDA'
get_stock_details_and_plot(symbol)

NVDA-基本資訊


4.公司財報

In [ ]:
# Function to fetch financial statements
def financial_statements(symbol):
    try:
        stock_info = yf.Ticker(symbol)
        balance_sheet = stock_info.balance_sheet
        income_statement = stock_info.financials
        cash_flow = stock_info.cashflow

        return balance_sheet, income_statement, cash_flow
    except Exception as e:
        print(f"獲取財務報表發生錯誤：{str(e)}")
        return None, None, None

# Function to display financial statements
def display_financial_statements(balance_sheet, income_statement, cash_flow):
    if balance_sheet is not None:
        print("\n資產負債表-年")
        print(balance_sheet)

    if income_statement is not None:
        print("\n綜合損益表-年")
        print(income_statement)

    if cash_flow is not None:
        print("\n現金流量表-年")
        print(cash_flow)

symbol = 'NVDA'
balance_sheet, income_statement, cash_flow = financial_statements(symbol)
display_financial_statements(balance_sheet, income_statement, cash_flow)


資產負債表-年
                                                     2024-01-31  \
Treasury Shares Number                                      NaN   
Ordinary Shares Number                            24640000000.0   
Share Issued                                      24640000000.0   
Net Debt                                           2429000000.0   
Total Debt                                        11056000000.0   
...                                                         ...   
Allowance For Doubtful Accounts Receivable                  NaN   
Gross Accounts Receivable                                   NaN   
Cash Cash Equivalents And Short Term Investments  25984000000.0   
Other Short Term Investments                      18704000000.0   
Cash And Cash Equivalents                          7280000000.0   

                                                     2023-01-31  \
Treasury Shares Number                                      NaN   
Ordinary Shares Number                            24

In [ ]:
# Function to fetch financial statements
def financial_statements_quarterly(symbol):
    try:
        stock_info = yf.Ticker(symbol)
        quarterly_balance_sheet = stock_info.quarterly_balance_sheet
        quarterly_income_statement = stock_info.quarterly_income_stmt
        quarterly_cash_flow = stock_info.quarterly_balance_sheet

        return quarterly_balance_sheet, quarterly_income_statement, quarterly_cash_flow
    except Exception as e:
        print(f"獲取財務報表發生錯誤：{str(e)}")
        return None, None, None

# Function to display financial statements
def display_financial_statements_quarterly(quarterly_balance_sheet, quarterly_income_statement, quarterly_cash_flow):
    if balance_sheet is not None:
        print("\n資產負債表-季")
        print(quarterly_balance_sheet)

    if income_statement is not None:
        print("\n綜合損益表-季")
        print(quarterly_balance_sheet)

    if cash_flow is not None:
        print("\n現金流量表-季")
        print(quarterly_cash_flow)

symbol = 'NVDA'
quarterly_balance_sheet, quarterly_income_statement, quarterly_cash_flow = financial_statements_quarterly(symbol)
display_financial_statements_quarterly(quarterly_balance_sheet, quarterly_income_statement, quarterly_cash_flow)


資產負債表-季
                                                     2024-07-31  \
Treasury Shares Number                                      NaN   
Ordinary Shares Number                            24562000000.0   
Share Issued                                      24562000000.0   
Net Debt                                                    NaN   
Total Debt                                        10015000000.0   
...                                                         ...   
Receivables                                       14132000000.0   
Accounts Receivable                               14132000000.0   
Cash Cash Equivalents And Short Term Investments  34800000000.0   
Other Short Term Investments                      26237000000.0   
Cash And Cash Equivalents                          8563000000.0   

                                                     2024-04-30  \
Treasury Shares Number                                      NaN   
Ordinary Shares Number                            24

5.交易數據

In [ ]:
# 獲取股票數據的函數
def get_stock_data(symbol, time_range):
    stock_data = yf.download(symbol, period=time_range)
    return stock_data

# 計算價格差異的函數
def calculate_price_difference(stock_data, period_days):
    latest_price = stock_data.iloc[-1]["Adj Close"]  # 獲取最新的收盤價
    previous_price = stock_data.iloc[-period_days]["Adj Close"] if len(stock_data) > period_days else stock_data.iloc[0]["Adj Close"]  # 獲取特定天數前的收盤價
    price_difference = latest_price - previous_price  # 計算價格差異
    percentage_difference = (price_difference / previous_price) * 100  # 計算百分比變化
    return price_difference, percentage_difference  # 返回價格差異和百分比變化

# 使用範例
symbol = 'NVDA'  # 蘋果公司的股票代碼
time_range = '1y'  # 時間範圍：1年

# 獲取股票數據
stock_data = get_stock_data(symbol, time_range)

if stock_data is not None and not stock_data.empty:
    period_days = 252  # 一年的交易日數
    price_difference, percentage_difference = calculate_price_difference(stock_data, period_days)

    latest_close_price = stock_data.iloc[-1]["Adj Close"]
    highest_price = stock_data["High"].max()
    lowest_price = stock_data["Low"].min()

    # 輸出數據
    print(f"{symbol} 最新收盤價: ${latest_close_price:.2f}")
    print(f"{symbol} {time_range} 增長率: ${price_difference:.2f}, 百分比: {percentage_difference:+.2f}%")
    print(f"{symbol} {time_range} 最高價: ${highest_price:.2f}")
    print(f"{symbol} {time_range} 最低價: ${lowest_price:.2f}")

    # 繪製 K 線圖
    fig = go.Figure()
    fig = plotly.subplots.make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.01, row_heights=[0.8, 0.5, 0.5, 0.5])

    mav5 = stock_data['Adj Close'].rolling(window=5).mean()  # 5日mav
    mav20 = stock_data['Adj Close'].rolling(window=20).mean()  # 20日mav
    mav60 = stock_data['Adj Close'].rolling(window=60).mean()  # 60日mav
    rsi = RSIIndicator(close=stock_data['Adj Close'], window=14)
    macd = MACD(close=stock_data['Adj Close'], window_slow=26, window_fast=12, window_sign=9)

    fig.add_trace(go.Candlestick(x=stock_data.index,
                                  open=stock_data['Open'],
                                  high=stock_data['High'],
                                  low=stock_data['Low'],
                                  close=stock_data['Adj Close']), row=1, col=1)

    fig.add_trace(go.Scatter(x=stock_data.index, y=mav5, opacity=0.7, line=dict(color='blue', width=2), name='MAV-5'), row=1, col=1)
    fig.add_trace(go.Scatter(x=stock_data.index, y=mav20, opacity=0.7, line=dict(color='orange', width=2), name='MAV-20'), row=1, col=1)
    fig.add_trace(go.Scatter(x=stock_data.index, y=mav60, opacity=0.7, line=dict(color='purple', width=2), name='MAV-60'), row=1, col=1)

    # 繪製交易量
    colors = ['green' if row['Open'] - row['Adj Close'] >= 0 else 'red' for index, row in stock_data.iterrows()]
    fig.add_trace(go.Bar(x=stock_data.index, y=stock_data['Volume'], marker_color=colors, name='Volume'), row=2, col=1)

    # 繪製 RSI
    fig.add_trace(go.Scatter(x=stock_data.index, y=rsi.rsi(), line=dict(color='purple', width=2)), row=3, col=1)
    fig.add_trace(go.Scatter(x=stock_data.index, y=[70]*len(stock_data.index), line=dict(color='red', width=1), name='Overbought'), row=3, col=1)
    fig.add_trace(go.Scatter(x=stock_data.index, y=[30]*len(stock_data.index), line=dict(color='green', width=1), name='Oversold'), row=3, col=1)

    # 繪製 MACD
    colorsM = ['green' if val >= 0 else 'red' for val in macd.macd_diff()]
    fig.add_trace(go.Bar(x=stock_data.index, y=macd.macd_diff(), marker_color=colorsM), row=4, col=1)
    fig.add_trace(go.Scatter(x=stock_data.index, y=macd.macd(), line=dict(color='orange', width=2)), row=4, col=1)
    fig.add_trace(go.Scatter(x=stock_data.index, y=macd.macd_signal(), line=dict(color='blue', width=1)), row=4, col=1)

    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="Volume", row=2, col=1)
    fig.update_yaxes(title_text="RSI", row=3, col=1)
    fig.update_yaxes(title_text="MACD", row=4, col=1)

    # 顯示圖表
    fig.show()
else:
    print(f'查無 {symbol} 數據')

[*********************100%***********************]  1 of 1 completed

NVDA 最新收盤價: $126.69
NVDA 1y 增長率: $84.23, 百分比: +198.41%
NVDA 1y 最高價: $140.76
NVDA 1y 最低價: $39.23


6.機構評級

In [ ]:
import requests as res
import pandas as pd
from bs4 import BeautifulSoup
import plotly.graph_objects as go
import plotly.express as px

def scrape_and_plot_finviz_data(symbol):
    # 爬蟲部分
    url = f"https://finviz.com/quote.ashx?t={symbol}"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = res.get(url, headers=headers)

    # 檢查請求是否成功
    if response.status_code != 200:
        raise Exception(f"無法從 {url} 獲取數據，狀態碼: {response.status_code}")

    soup = BeautifulSoup(response.content, 'html.parser')

    # 定位包含分析師評級的表格
    table = soup.find('table', class_='js-table-ratings styled-table-new is-rounded is-small')

    # 檢查是否成功找到表格
    if table is None:
        raise Exception("未能在頁面上找到評級表格。")

    # 從表格中提取數據
    data = []
    for row in table.find_all('tr')[1:]:  # 跳過表頭
        cols = row.find_all('td')
        data.append({
            "Date": cols[0].text.strip(),
            "Action": cols[1].text.strip(),
            "Analyst": cols[2].text.strip(),
            "Rating Change": cols[3].text.strip(),
            "Price Target Change": cols[4].text.strip() if len(cols) > 4 else None
        })

    # 將數據轉換為 DataFrame
    df = pd.DataFrame(data)

    # 移除空的目標價格變化
    df = df.dropna(subset=['Price Target Change'])

    # 清理數據，替換特殊字符
    df['Price Target Change'] = df['Price Target Change'].str.replace('→', '->').str.replace(' ', '')

    # 將目標價格變化轉換為數值範圍
    price_change_ranges = df['Price Target Change'].str.extract(r'\$(\d+)->\$(\d+)')
    price_change_ranges = price_change_ranges.apply(pd.to_numeric)
    df['Price Target Start'] = price_change_ranges[0]
    df['Price Target End'] = price_change_ranges[1]

    # 動態生成評級變化的順序
    rating_order = df['Rating Change'].unique().tolist()
    df['Rating Change'] = pd.Categorical(df['Rating Change'], categories=rating_order, ordered=True)

    # 可視化 1：分析師的目標價格變化
    fig1 = go.Figure()
    for i, row in df.iterrows():
        fig1.add_trace(go.Scatter(
            x=[row['Price Target Start'], row['Price Target End']],
            y=[row['Analyst'], row['Analyst']],
            mode='lines+markers+text',
            line=dict(color='blue' if row['Price Target End'] >= row['Price Target Start'] else 'red', width=2),
            marker=dict(size=10, color='blue' if row['Price Target End'] >= row['Price Target Start'] else 'red'),
            text=[f"${row['Price Target Start']}", f"${row['Price Target End']}"],
            textposition="top center"
        ))

    fig1.update_layout(
        title='機構目標價格變化',
        xaxis_title='目標價格',
        yaxis_title='機構',
        yaxis=dict(type='category'),
        showlegend=False,
        height=800,  # 增加圖表高度
        width=1200   # 增加圖表寬度
    )

    # 可視化 2：評級變化的分佈，使用不同顏色
    fig2 = px.histogram(df,
                         x='Rating Change',
                         title='機構評級變化分佈',
                         color='Rating Change',
                         category_orders={'Rating Change': rating_order})  # 明確排序順序

    fig2.update_layout(
        height=800,  # 增加圖表高度
        width=1200   # 增加圖表寬度
    )

    # 顯示圖表
    fig1.show()
    fig2.show()

    # 顯示評級數據
    print(df)

# 使用示例
symbol = 'NVDA'  # 替換為所需的股票符號
scrape_and_plot_finviz_data(symbol)

         Date      Action                  Analyst  Rating Change  \
0   Sep-18-24   Initiated            William Blair     Outperform   
1   Aug-29-24  Reiterated                  Needham            Buy   
2   Aug-19-24  Reiterated                  Goldman            Buy   
3   Aug-06-24     Upgrade               New Street  Neutral → Buy   
4   Jul-15-24  Reiterated                 TD Cowen            Buy   
5   Jul-12-24  Reiterated    The Benchmark Company            Buy   
6   Jul-09-24  Reiterated  KeyBanc Capital Markets     Overweight   
7   Jul-08-24  Reiterated                      UBS            Buy   
8   Jul-05-24   Downgrade               New Street  Buy → Neutral   
9   Jul-01-24  Reiterated           Morgan Stanley     Overweight   
10  Jun-18-24  Reiterated               Rosenblatt            Buy   
11  Jun-03-24  Reiterated          BofA Securities            Buy   
12  May-24-24   Downgrade                  DZ Bank     Buy → Hold   
13  May-23-24  Reiterated         

7.相關新聞

In [ ]:
def get_stock_news(symbol):
    url = f"https://finviz.com/quote.ashx?t={symbol}&p=d"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}

    try:
        response = res.get(url, headers=headers)
        response.raise_for_status()  # 確保請求成功
    except res.exceptions.RequestException as e:
        print(f"無法獲取 {symbol} 相關消息: {e}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')

    # 找到所有新聞項目
    news_table = soup.find('table', class_='fullview-news-outer')
    if news_table is None:
        print(f"無法獲取 {symbol} 相關新聞表格")
        return None

    news_items = news_table.find_all('tr')
    news_data = []

    for news_item in news_items:
        cells = news_item.find_all('td')
        if len(cells) < 2:
            continue

        date_info = cells[0].text.strip()
        news_link = cells[1].find('a', class_='tab-link-news')
        if news_link:
            news_title = news_link.text.strip()
            news_url = news_link['href']
            news_data.append({'Date': date_info, 'Title': news_title, 'URL': news_url})

    return news_data

# 使用示例
symbol = 'NVDA'
news_data = get_stock_news(symbol)

if news_data:
    # 將新聞數據轉換為 DataFrame
    df = pd.DataFrame(news_data)
    print(f"{symbol} - 近期相關消息")
    print(df)  # 顯示表格

    # 打印所有新聞連結
    print(f"\n展開 {symbol} - 近期相關消息連結:")
    for news in news_data:
        print(f"**[{news['Title']}]({news['URL']})**")
    print(f"[資料來源](https://finviz.com/quote.ashx?t={symbol})")
else:
    print(f"查無 {symbol} 近期相關消息")